In [45]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever

In [59]:
import os
os.environ["OPENAI_API_KEY"]

sk-ElxqY36z2QziARK2FGZw2eSxuuZGyYU1-a1lLjt-_fT3BlbkFJUF06ZwtOrqItoq0ZZhZEN6E-lOz1P-Kbm7Q75Ru5MA


In [47]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=500)

In [48]:
## Carregar o PDF
pdf_link = "confeccao_atas.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

In [49]:
len(pages)

18

In [50]:
## Splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)
parent_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 4000,
  chunk_overlap = 200,
  length_function = len,
  add_start_index = True
)

In [51]:
## Storages
store = InMemoryStore()
vector_store = Chroma(embedding_function=embeddings, persist_directory='child_vectordb')

In [52]:
parent_document_retriever = ParentDocumentRetriever(
  vectorstore=vector_store,
  docstore=store,
  child_splitter=child_splitter,
  parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages, ids=None)

In [53]:
parent_document_retriever.vectorstore.get()

{'ids': ['00b4a8af-be8d-44e7-9c26-bb2f51b7d4ec',
  '020520d3-0c68-4f44-8dbe-d48883fa1ebd',
  '02806ece-a4ab-419f-9ac5-c1f15a8def65',
  '02ad2577-120f-41c0-ac42-7d257df22ffa',
  '02bc6cb3-3827-4590-8ec6-f11f228cf797',
  '02d9b234-0984-4f9d-b9e0-a8f042daf7ac',
  '0346f612-7c98-416e-b31a-3086e0554320',
  '0386efd2-0d3a-404f-92d6-339564042faa',
  '040c394a-75e9-4003-bed8-45a0768cc11e',
  '0416433c-d295-4be8-ba87-d07c27255ae7',
  '041c824d-d203-4e20-ad5b-4c2e04da9f77',
  '044c4d5d-04a3-4263-88cf-78e98f6f1b80',
  '048bb5dd-09c4-4d9f-96ff-305c9f76819e',
  '048c6727-a9a7-49eb-b9d8-efba459c46f0',
  '049c7b5d-5298-4b8a-ba8e-9ac098bcdec3',
  '051f3d94-647d-4dc3-84e4-1ff09542398d',
  '05758d71-78a9-4591-ae16-1f85260fafb0',
  '0609eec6-6f7b-4ba4-9398-6283853e3552',
  '0641adf1-466d-4b5e-b18a-8dd6339a13b3',
  '065f27ec-c329-46c1-8949-ec7d9162105e',
  '06700cb1-4ab5-4c19-8119-7c9177f44b76',
  '06a9129d-5b0b-41eb-b026-ae7d495d02c7',
  '06b6cd25-7d93-428b-b991-5559f87b4c58',
  '06f64c48-2e5e-4b48-b44e-

In [54]:
## Template de Prompt
TEMPLATE = """"
  Você é um especialista em atas de reunião e está conversando com um colega que está tendo dificuldades em entender o conteúdo de uma ata.

  Query:
  {question}  

  Context:
  {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)


In [55]:
setup_retrieval = RunnableParallel({
  "question": RunnablePassthrough(), 
  "context": parent_document_retriever
})

output_parser = StrOutputParser()

In [56]:
parent_chain_retrieval = setup_retrieval | rag_prompt | llm | output_parser

In [58]:
parent_chain_retrieval.invoke("Como lavrar um ata?")

'Para lavrar uma ata corretamente, é importante seguir algumas diretrizes, como:\n\n1. Escrever a ata sem entrelinhas, emendas ou rasuras.\n2. Utilizar abreviações apenas para títulos, tratamentos de deferência e expressões consagradas pelo uso geral.\n3. Em caso de engano, lapso de linguagem ou omissão, é possível fazer uma correção através de um auto de correção, emenda ou acréscimo.\n4. Se o presidente do Conselho acumular as funções de Secretário, deve acrescentar as palavras "Presidente-Secretário" em sua assinatura.\n5. Após a última ata lavrada no livro do Conselho, antes da reunião ordinária do Presbitério, deve-se fazer o registro da estatística do movimento espiritual e financeiro de cada ano.\n6. O texto da ata deve ser dividido em itens bem definidos, permitindo uma melhor utilização dos recursos de formatação de caractere (maiúsculas, negrito, itálico e sublinhado) para destacar os assuntos mais importantes e facilitar a pesquisa posterior de informações no texto.'